In [3]:
'''
Wrangling npz files:
http://stackoverflow.com/questions/32682928/loading-arrays-from-npz-files-in-pythhon'''

import cv2
import numpy as np
# np.set_printoptions(threshold=np.nan)
import pandas as pd
import matplotlib.pyplot as plt
import glob
import tqdm
from collections import deque

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, f_regression
from sklearn import preprocessing

In [2]:
# Flip the LEFT & RIGHT directional inputs, but leave FORWARD alone. (for directional target/label data)
# array([[ 1.,  0.,  0.],   <-- FORWARD-LEFT
#        [ 0.,  1.,  0.],   <-- FORWARD-RIGHT
#        [ 0.,  0.,  1.]])  <-- FORWARD

def flip_dir(array):

    array_list = array.tolist()

    final = []
    
    for row in array_list:
        
        # Forward-Left becomes Forward-Right
        if row[0] == 1:
            row = [0,1,0]
            final.append(row)
            
        # Forward-Right becomes Forward-Left
        elif row[1] == 1:
            row = [1,0,0]
            final.append(row)
            
        # Forward remains as Forward
        elif row[2] == 1:
            final.append(row)
            continue
    
    final = np.array(final)       
#     print final
#     print type(final)
    return final

In [3]:
# Unflatten (for image feature data)
def unflatten_flip(array,width,height):
    
    final = []
    
    # Convert the nparray to a list; pull the list from the outer set of square brackets; deque the list.
#     array_list = array.tolist()

#     print 'This is ARRAY: ',array

#     for a in array:
#         a = list(a)
#         print 'THIS IS a: ', a
#         initial = deque(a)
    
    
    initial = deque(array)
    
    # For each row in the specified height dimension...
    for row in xrange(height):
        temp_row = []

        # ...for each column in the specified width dimension...
        for col in xrange(width):
            
            # ...pop off the last item in the initial array and append to the new temp_row.
            d = initial.pop()
            temp_row.append(d)
            
        # Append temp_row to final, and reset temp_row to empty.
        final.append(temp_row)
        temp_row = []

    # Now we have the final list, but it's 'upside down' because we took care of left to right, but not bottom to top.
    final = deque(final)
    
    # This takes care of bottom to top.
    final.reverse()
   
    final = np.array(final)       
#     print final
#     print type(final)
    return final

In [8]:
# load training data
image_array = np.zeros((1, 38400))
label_array = np.zeros((1, 3), 'float')
training_data = glob.glob('training_data/testRT4.npz')        # Finds filenames matching specified path or pattern.

for single_npz in training_data:                        
    with np.load(single_npz) as data:
#         print data.keys()
        
        # LABELS (this works)        
        labels = data.f.train_labels
        print 'Original Labels dims: ', labels.shape
        
        labels_flipped = flip_dir(labels)
        
        # FEATURES (this works)
        image_array = np.zeros((1, 38400))
        
        original_features = data.f.train
        print 'Original Features dims: ', original_features.shape
        
        for feature in original_features:
            
#             print 'This is the ORIGINAL feature: ', type(feature), feature.shape, feature
            feature = list(feature)
            
            feature_array_flipped = unflatten_flip(feature,width=320,height=120)
            
            temp_array = feature_array_flipped.reshape(1, 38400).astype(np.float32)
#             print 'This is the FLIPPED feature: ', type(temp_array), temp_array.shape, temp_array 
            
            image_array = np.vstack((image_array, temp_array))
            
        train_labels = labels_flipped        
        train = image_array[1:, :]

        print 'Flipped Labels dims: ', train_labels.shape
        print 'Flipped Features dims: ', train.shape        
        
        np.savez('training_data_temp/testFLIPPED.npz', train=train, train_labels=train_labels)      
        
        print 'Flipping is complete.'

Original Labels dims:  (184, 3)
Original Features dims:  (184, 38400)
Flipped Labels dims:  (184, 3)
Flipped Features dims:  (184, 38400)
Flipping is complete.


In [4]:
'''Need a subsetted sample of just the directional label_array'''

training_data = glob.glob('training_data/testRT4.npz') 

for single_npz in training_data:                        
    with np.load(single_npz) as data:
#         print data.keys()
        
        # LABELS (this works)        
        labels = data.f.train_labels
        print 'Original Labels dims: ', labels.shape
        
        subset = labels[0:101]
        print 'Subset Labels dims: ', subset.shape
        np.savez('training_images/label_array_SUBSET.npz', train_labels=subset)
        

Original Labels dims:  (184, 3)
Subset Labels dims:  (101, 3)


In [14]:
'''Updating a log (txt file) of training data activities.'''

import time

total_frame = 1000
saved_frame = 399
clicks_forward = 101
clicks_forward_left = 102
clicks_forward_right = 104

# 'a' is for append
with open("img_collect_log.txt", "a") as f:

    f.write("Date: "  + time.strftime("%x") + "\n")
    f.write("Time: "  + time.strftime("%X") + "\n")
    f.write("Total frames: " + str(total_frame) + "\n")
    f.write("Saved frames: " + str(saved_frame) + "\n")
    f.write("Dropped frames: " + str(total_frame - saved_frame) + "\n")
    f.write("Forward clicks: " + str(clicks_forward) + "\n")
    f.write("Forward-left clicks: " + str(clicks_forward_left) + "\n")
    f.write("Forward-right clicks: " + str(clicks_forward_right) + "\n")
    f.write("-----------------------------\n")


In [10]:
from tqdm import tqdm

label_array = np.zeros((1, 3), 'float')

original_labels_shape = None
labels_doubled = None
print 'all good up to here'

training_data_labels = glob.glob('./imgs_2016*/label_array_ORIGINALS.npz')
print len(training_data_labels)

pbar = tqdm(total=100)

for single_npz in training_data_labels:
    print single_npz

    with np.load(single_npz) as data:

        # LABELS
        labels = data.f.train_labels
        original_labels_shape = labels.shape
        print 'Original Labels dims: ', labels.shape

        array_list = labels.tolist()

        final = []

        for row in array_list:

            # First append original row to final, then...
            final.append(row)

            # Forward-Left becomes Forward-Right, append
            if row[0] == 1:
                row = [0,1,0]
                final.append(row)

            # Forward-Right becomes Forward-Left, append
            elif row[1] == 1:
                row = [1,0,0]
                final.append(row)

            # Forward remains as Forward, append
            elif row[2] == 1:
                final.append(row)
                continue

        # Convert final to np.array
        labels_doubled = np.array(final)
        print 'Original + Flipped Labels dims: ', labels_doubled.shape

        # vstack 'labels_doubled' onto 'label_array', from below
        label_array = np.vstack((label_array, labels_doubled))
        pbar.update(100/len(training_data_labels))

# save training images and labels (selects row 1 on down because the row 0 is just zeros)

train_labels = label_array[1:, :]

pbar.close()

 96%|█████████▌| 96/100 [00:00<00:00, 2999.93it/s]

all good up to here
16
./imgs_20161024_201843/label_array_ORIGINALS.npz
Original Labels dims:  (140, 3)
Original + Flipped Labels dims:  (280, 3)
./imgs_20161024_203956/label_array_ORIGINALS.npz
Original Labels dims:  (150, 3)
Original + Flipped Labels dims:  (300, 3)
./imgs_20161024_205205/label_array_ORIGINALS.npz
Original Labels dims:  (153, 3)
Original + Flipped Labels dims:  (306, 3)
./imgs_20161024_210342/label_array_ORIGINALS.npz
Original Labels dims:  (155, 3)
Original + Flipped Labels dims:  (310, 3)
./imgs_20161024_213033/label_array_ORIGINALS.npz
Original Labels dims:  (157, 3)
Original + Flipped Labels dims:  (314, 3)
./imgs_20161024_213758/label_array_ORIGINALS.npz
Original Labels dims:  (158, 3)
Original + Flipped Labels dims:  (316, 3)
./imgs_20161024_214826/label_array_ORIGINALS.npz
Original Labels dims:  (155, 3)
Original + Flipped Labels dims:  (310, 3)
./imgs_20161024_215723/label_array_ORIGINALS.npz
Original Labels dims:  (160, 3)
Original + Flipped Labels dims:  (3

In [11]:
from tqdm import tqdm

label_array = np.zeros((1, 3), 'float')

original_labels_shape = None
labels_doubled = None
print 'all good up to here'

training_data_labels = glob.glob('./imgs_2016*/label_array_ORIGINALS.npz')
print len(training_data_labels)


for single_npz in tqdm(training_data_labels):
    print single_npz

    with np.load(single_npz) as data:

        # LABELS
        labels = data.f.train_labels
        original_labels_shape = labels.shape
        print 'Original Labels dims: ', labels.shape

        array_list = labels.tolist()

        final = []

        for row in array_list:

            # First append original row to final, then...
            final.append(row)

            # Forward-Left becomes Forward-Right, append
            if row[0] == 1:
                row = [0,1,0]
                final.append(row)

            # Forward-Right becomes Forward-Left, append
            elif row[1] == 1:
                row = [1,0,0]
                final.append(row)

            # Forward remains as Forward, append
            elif row[2] == 1:
                final.append(row)
                continue

        # Convert final to np.array
        labels_doubled = np.array(final)
        print 'Original + Flipped Labels dims: ', labels_doubled.shape

        # vstack 'labels_doubled' onto 'label_array', from below
        label_array = np.vstack((label_array, labels_doubled))

# save training images and labels (selects row 1 on down because the row 0 is just zeros)

train_labels = label_array[1:, :]


100%|██████████| 16/16 [00:00<00:00, 507.52it/s]

all good up to here
16
./imgs_20161024_201843/label_array_ORIGINALS.npz
Original Labels dims:  (140, 3)
Original + Flipped Labels dims:  (280, 3)
./imgs_20161024_203956/label_array_ORIGINALS.npz
Original Labels dims:  (150, 3)
Original + Flipped Labels dims:  (300, 3)
./imgs_20161024_205205/label_array_ORIGINALS.npz
Original Labels dims:  (153, 3)
Original + Flipped Labels dims:  (306, 3)
./imgs_20161024_210342/label_array_ORIGINALS.npz
Original Labels dims:  (155, 3)
Original + Flipped Labels dims:  (310, 3)
./imgs_20161024_213033/label_array_ORIGINALS.npz
Original Labels dims:  (157, 3)
Original + Flipped Labels dims:  (314, 3)
./imgs_20161024_213758/label_array_ORIGINALS.npz
Original Labels dims:  (158, 3)
Original + Flipped Labels dims:  (316, 3)
./imgs_20161024_214826/label_array_ORIGINALS.npz
Original Labels dims:  (155, 3)
Original + Flipped Labels dims:  (310, 3)
./imgs_20161024_215723/label_array_ORIGINALS.npz
Original Labels dims:  (160, 3)
Original + Flipped Labels dims:  (3

In [14]:
import csv

npzfilename = 'TrentReznor'
rotation_range = 1
width_shift_range = 0.02
height_shift_range = 0.02
shear_range = 0
zoom_range = 0.1
fill_mode = 'nearest'
sigma = 0.33

row_params = [npzfilename, \
             rotation_range, \
             width_shift_range, \
             height_shift_range, \
             shear_range, \
             zoom_range, \
             fill_mode, \
             sigma]


with open('../parameter_log.csv','a') as log:
    log_writer = csv.writer(log)
    log_writer.writerow(row_params)


u'/Users/PaulYim/Desktop/car/computer/notebooks'